In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.all import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *
from local.metrics import *
from local.text.core import *
from local.text.data import *
from local.text.models.core import *
from local.text.models.awdlstm import *
from local.text.learner import *
from local.callback.rnn import *
from local.callback.all import *

# Transfer learning in text

> How to fine-tune a language model and train a classifier

## Finetune a pretrained Language Model

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')
df_tok,count = tokenize_df(df, 'text')
texts,lengths = df_tok['text'],df_tok['text_lengths'].values.astype(np.int)

splits = RandomSplitter()(texts)
vocab = make_vocab(count)
dsrc = DataSource(texts, [Numericalize(vocab)], filts=splits)

In [ ]:
dbunch = LMDataLoader.dbunchify(dsrc, bs=64, seq_len=72)

In [ ]:
learn = language_model_learner(dbunch, AWD_LSTM, vocab, metrics=[accuracy, Perplexity()], path=path)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-2, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.795627,4.312182,0.260525,74.603088,00:04
1,4.770514,4.284557,0.262117,72.570419,00:04
2,4.752245,4.273811,0.263455,71.794716,00:04
3,4.747759,4.272093,0.263581,71.671509,00:04


In [ ]:
learn.save_encoder('enc')

## Use it to train a classifier

In [ ]:
splits = RandomSplitter()(range_of(df_tok))
dsrc = DataSource(df_tok, filts=splits, tfms=[
    [attrgetter("text"), Numericalize(vocab)],
    [attrgetter("label"), Categorize()]])
trn_dl = SortedDL(dsrc.train, create_batch=pad_collate, after_batch=[Cuda], shuffle=True, drop_last=True)
val_dl = SortedDL(dsrc.valid, create_batch=pad_collate, after_batch=[Cuda])
dbunch = DataBunch(trn_dl, val_dl)

In [ ]:
learn = text_classifier_learner(dbunch, AWD_LSTM, vocab, metrics=[accuracy], path=path)

In [ ]:
learn = learn.load_encoder('enc')

In [ ]:
learn.fit_one_cycle(4, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.849841,0.601815,0.675000,00:06
1,0.753493,0.568468,0.720000,00:06
2,0.753582,0.548259,0.745000,00:06
3,0.740121,0.533096,0.745000,00:06


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(8, np.array([1e-5,3e-4,1e-4,3e-3,1e-3]), moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.710222,0.536359,0.750000,00:10
1,0.715015,0.531864,0.745000,00:10
2,0.737472,0.531057,0.750000,00:09
3,0.682374,0.526255,0.745000,00:10
4,0.707951,0.531820,0.735000,00:10
5,0.723326,0.524388,0.760000,00:10
6,0.685744,0.527304,0.745000,00:10
7,0.732956,0.520143,0.755000,00:10
